In [1]:
import os
import glob
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
import sys
sys.path.insert(1, 'E:/linux/models/research')

In [ ]:
from object_detection.utils import dataset_util

In [2]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import io
import PIL.Image
import random

In [3]:
annotation_dir = "E:\JupyterNotebook\dataset\Annotation"
image_dir = "E:\JupyterNotebook\dataset\Images"

In [4]:
output_dir = "E:\JupyterNotebook\dataset"

In [5]:
def get_image_path(annotation_path):
    path_items = annotation_path.split(os.path.sep)
    return os.path.join(image_dir, path_items[-2], path_items[-1] + '.jpg')

In [ ]:
def get_encoded_image(image_path, desire_format = 'JPEG'):
    with tf.gfile.GFile(image_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = PIL.Image.open(encoded_jpg_io)
    if image.format != desire_format:
        raise ValueError('Image format not JPEG: {}'.format(image_path))
    return encoded_jpg

In [ ]:
def create_tf_example(annotation_path):
    xml = ET.parse(annotation_path).getroot()
    filename = xml.find("filename").text
    source = xml.find("source").find("database").text
    size = xml.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)
    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)
    image_path = get_image_path(annotation_path)
    encoded_jpg = get_encoded_image(image_path)
    
    for obj in xml.findall("object"):
        bndbox = obj.find("bndbox")
        xmin, ymin, xmax, ymax = bndbox.find("xmin"), bndbox.find("ymin"), bndbox.find("xmax"), bndbox.find("ymax")
        xmin, ymin, xmax, ymax = float(xmin.text), float(ymin.text), float(xmax.text), float(ymax.text)
        xmins.append(xmin / width)
        ymins.append(ymin / height)
        xmaxs.append(xmax / width)
        ymaxs.append(ymax / height)
        classes_text.append('dog'.encode('utf8'))
        classes.append(1) #Only one class
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(source.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [ ]:
all_annotation_files = glob.glob(r'E:\JupyterNotebook\dataset\Annotation\**\*', recursive = True)
all_annotation_files = [f for f in all_annotation_files if os.path.isfile(f)]

In [ ]:
dataset = random.sample(all_annotation_files, 3000)

In [ ]:
#Ensure all image are in jpeg format
for ant in dataset:
    img_path = get_image_path(ant)
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    if len(img.shape) != 3 or img.shape[-1] != 3:
        print(img_path)
        img_bgr = cv2.imread(img_path)
        cv2.imwrite(img_path, img_bgr)

In [ ]:
train_ds = dataset[: 2000]
eval_ds = dataset[2000: 2500]
test_ds = dataset[2500: ]

In [ ]:
print(len(train_ds), len(eval_ds), len(test_ds))

In [ ]:
all_ds = [train_ds, eval_ds, test_ds]
ds_files = ["train_ds.tfrecord", "eval_ds.tfrecord", "test_ds.tfrecord"]
for (ds, ds_file) in zip(all_ds, ds_files):
    with tf.python_io.TFRecordWriter(os.path.join(output_dir, ds_file)) as writer:
        for ant in ds:
            example = create_tf_example(ant)
            writer.write(example.SerializeToString())

# Create CSV file

In [6]:
images_dir = r"E:\JupyterNotebook\dataset\dataset"

In [7]:
def get_labels(annotation_path):
    list_bndboxes = []
    xml = ET.parse(annotation_path).getroot()
    filename = os.path.basename(get_image_path(annotation_path))
    for obj in xml.findall("object"):
        bndbox = obj.find("bndbox")
        xmin, ymin, xmax, ymax = bndbox.find("xmin"), bndbox.find("ymin"), bndbox.find("xmax"), bndbox.find("ymax")
        xmin, ymin, xmax, ymax = float(xmin.text), float(ymin.text), float(xmax.text), float(ymax.text)
        list_bndboxes.append([filename, xmin, xmax, ymin, ymax, 1])
    return list_bndboxes

In [8]:
all_annotation_files = glob.glob(r'E:\JupyterNotebook\dataset\Annotation\**\*', recursive = True)
all_annotation_files = [f for f in all_annotation_files if os.path.isfile(f)]

In [9]:
dataset = random.sample(all_annotation_files, 3000)

In [15]:
#Ensure all image are in jpeg format
for ant in dataset:
    img_path = get_image_path(ant)
    img = cv2.imread(img_path)
    cv2.imwrite(os.path.join(images_dir, os.path.basename(img_path)), img)

In [11]:
train_ds = dataset[: 2000]
eval_ds = dataset[2000: 2500]
test_ds = dataset[2500: ]

In [12]:
import csv

In [13]:
label_files = ['labels_train.csv', 'labels_val.csv', 'labels_test.csv']
ds = [train_ds, eval_ds, test_ds]

In [14]:
for (label_file, d) in zip(label_files, ds):
    labels = []
    for ant in d:
        boxes = get_labels(ant)
        labels.extend(boxes)
    with open(label_file, 'w', newline='') as csvFile:
        writer = csv.writer(csvFile, delimiter=',')
        writer.writerows(labels)